# DS 2002 - Project 1
Christine Tsai (rtg7bs)

### Some notes about this project:
- My data was originally all in one CSV file, so I was unable to fully follow the path of exporting the data for one of my tables into a JSON as explained in the approach when using an existing OLTP database in the email Professor Tupitza sent on 10/31/24


## 1.0 Extract Phase: Load Data from CSV File

In [12]:
# imports
import os
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, text
import json
import numpy
import datetime
import certifi
import pymongo



In [13]:
# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    "user_name" : "root",
    "password" : "example",
    #"cluster_name" : "cluster_0",
    #"cluster_subnet" : "cfyrc",
    "cluster_location" : "local", # "local"
    "db_name" : "hotel_booking"
}

### Code from Lab 4 Relating to MongoDB

In [14]:

def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe
def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe

In [15]:
# read in original source of data: csv file
data_dir = os.path.join(os.getcwd(), 'data')
data_file = os.path.join(data_dir, 'hotel_booking.csv')

df = pd.read_csv(data_file, header=0, index_col=0)
df.head()

,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,email,phone-number,credit_card
hotel,,,,,,,,,,,,,,,,,,,,,
Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,...,Transient,0.0,0,0,Check-Out,2015-07-01,Ernest Barnes,Ernest.Barnes31@outlook.com,669-792-1661,************4322
Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,...,Transient,0.0,0,0,Check-Out,2015-07-01,Andrea Baker,Andrea_Baker94@aol.com,858-637-6955,************9157
Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,...,Transient,75.0,0,0,Check-Out,2015-07-02,Rebecca Parker,Rebecca_Parker@comcast.net,652-885-2745,************3734
Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,...,Transient,75.0,0,0,Check-Out,2015-07-02,Laura Murray,Laura_M@gmail.com,364-656-8427,************5677
Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,...,Transient,98.0,0,1,Check-Out,2015-07-03,Linda Hines,LHines@verizon.com,713-226-5883,************5498


### 1.1 Drop Unimportant Fields

In [16]:
#df.drop(['required_car_parking_spaces','total_of_special_requests', 'adults', 'children', 'babies', 'meal', 'market_segment', 'distribution_channel'], axis=1, inplace=True)
df.columns


Index(['is_canceled', 'lead_time', 'arrival_date_year', 'arrival_date_month',
       'arrival_date_week_number', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', 'name', 'email',
       'phone-number', 'credit_card'],
      dtype='object')

## 2.0 Transform Phase: Split Data Into Dimension and Fact Tables

### Create `dim_guest` dataframe

In [17]:
dim_guest = df[['name', 'email', 'phone-number', 'credit_card', 'customer_type', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled']]

# insert new primary key col w/ incrementing val
dim_guest.insert(0, 'guest_id', range(1, len(dim_guest) + 1))
dim_guest.head()

,guest_id,name,email,phone-number,credit_card,customer_type,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled
hotel,,,,,,,,,
Resort Hotel,1,Ernest Barnes,Ernest.Barnes31@outlook.com,669-792-1661,************4322,Transient,0,0,0
Resort Hotel,2,Andrea Baker,Andrea_Baker94@aol.com,858-637-6955,************9157,Transient,0,0,0
Resort Hotel,3,Rebecca Parker,Rebecca_Parker@comcast.net,652-885-2745,************3734,Transient,0,0,0
Resort Hotel,4,Laura Murray,Laura_M@gmail.com,364-656-8427,************5677,Transient,0,0,0
Resort Hotel,5,Linda Hines,LHines@verizon.com,713-226-5883,************5498,Transient,0,0,0


### Export Guest Data into JSON
#### This step is extraneous for my particular approach when working exclusively with CSVs, but I included the steps necessary to export a file into a MongoDB collection to illustrate its functionality. 

In [18]:
# convert guest dataframe to json and export to mongodb
dim_guest.to_json('data/guest.json', orient = 'records', compression = 'infer')

client = get_mongo_client(**mongodb_args)
data_dir = os.path.join(os.getcwd(), 'data')

json_files = {"guests": "guest.json"}

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)  


In [19]:
# extract data from src mongodb collections into dataframes
client = get_mongo_client(**mongodb_args)
query = {}
collection = "guests"

df_guests = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_guests.head(2)

,guest_id,name,email,phone-number,credit_card,customer_type,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled
0,1,Ernest Barnes,Ernest.Barnes31@outlook.com,669-792-1661,************4322,Transient,0,0,0
1,2,Andrea Baker,Andrea_Baker94@aol.com,858-637-6955,************9157,Transient,0,0,0


### Create `dim_date` dataframe

In [20]:
dim_date = df[['arrival_date_day_of_month', 'arrival_date_week_number', 'arrival_date_month', 'arrival_date_year']]


# insert new full date col
dim_date['full_date'] = pd.to_datetime(
    dim_date['arrival_date_year'].astype(str) + '-' + 
    dim_date['arrival_date_month'].astype(str) + '-' + 
    dim_date['arrival_date_day_of_month'].astype(str)
)
dim_date.full_date = dim_date.full_date.astype('datetime64[ns]').dt.date

# remove duplicates so each unique full_date is associated w/ only 1 primary key
dim_date = dim_date.drop_duplicates(subset='full_date')

full_date_col = dim_date.pop('full_date')  # Remove the column
dim_date.insert(1, 'full_date', full_date_col) 

# insert new primary key col w/ incrementing val
dim_date.insert(0, 'date_id', range(1, len(dim_date) + 1))
dim_date.head()

/var/folders/xk/8b3j7bgd4r76vcr32987sg4r0000gn/T/ipykernel_3262/3378210570.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_date['full_date'] = pd.to_datetime(
/var/folders/xk/8b3j7bgd4r76vcr32987sg4r0000gn/T/ipykernel_3262/3378210570.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_date.full_date = dim_date.full_date.astype('datetime64[ns]').dt.date


,date_id,arrival_date_day_of_month,full_date,arrival_date_week_number,arrival_date_month,arrival_date_year
hotel,,,,,,
Resort Hotel,1,1,2015-07-01,27,July,2015
Resort Hotel,2,2,2015-07-02,27,July,2015
Resort Hotel,3,3,2015-07-03,27,July,2015
Resort Hotel,4,4,2015-07-04,27,July,2015
Resort Hotel,5,5,2015-07-05,28,July,2015


### Create `dim_reservation_status` dataframe

In [21]:
dim_reservation_status = df[['reservation_status']]

# remove duplicates so each unique combo is associated w/ only 1 primary key
dim_reservation_status = dim_reservation_status.drop_duplicates(subset='reservation_status')

# insert new primary key col w/ incrementing val
dim_reservation_status.insert(0, 'reservation_status_id', range(1, len(dim_reservation_status) + 1))
dim_reservation_status.head()

,reservation_status_id,reservation_status
hotel,,
Resort Hotel,1,Check-Out
Resort Hotel,2,Canceled
Resort Hotel,3,No-Show


### Create `fact_booking` dataframe

In [22]:
fact_booking = df[['name', 'reservation_status', 'reserved_room_type', 'assigned_room_type', 'arrival_date_day_of_month', 'arrival_date_month', 'arrival_date_year', 'is_canceled', 'lead_time', 'booking_changes', 'adr', 'stays_in_weekend_nights', 'stays_in_week_nights']]
# create dictionary of each full_date so we can look up what each full date's associated primary key in the dim_date table is
date_mapping = dim_date.set_index('full_date')['date_id'].to_dict()

# add new column arrival_date
fact_booking['arrival_date'] = pd.to_datetime(
    fact_booking['arrival_date_year'].astype(str) + '-' + 
    fact_booking['arrival_date_month'].astype(str) + '-' + 
    fact_booking['arrival_date_day_of_month'].astype(str)
)
# create mapping for each unique full date to assoicate w/ 1 date_id
fact_booking['date_id'] = fact_booking['arrival_date'].map(date_mapping)

# drop the date-related columns
fact_booking = fact_booking.drop(['arrival_date_day_of_month', 'arrival_date_month', 'arrival_date_year', 'arrival_date'], axis=1)

# ------ HANDLE RESERVATION STATUS 
# create dictionary 
res_status_mapping = dim_reservation_status.set_index('reservation_status')['reservation_status_id'].to_dict()

# create mapping for each unique reserved+assigned composite key to assoicate w/ 1 room_id
fact_booking['reservation_status_id'] = fact_booking['reservation_status'].map(res_status_mapping)

# drop reservation status column
fact_booking = fact_booking.drop('reservation_status', axis=1)

# ----- HANDLE GUEST
# create dict for mapping name to guest_id
guest_mapping = dim_guest.set_index('name')['guest_id'].to_dict()

fact_booking['guest_id'] = fact_booking['name'].map(guest_mapping)

# drop reservation status column
fact_booking = fact_booking.drop('name', axis=1)


# rearrange cols
date_id_col = fact_booking.pop('date_id')
fact_booking.insert(0, 'date_id', date_id_col)

# insert primary key col
fact_booking.insert(0, 'booking_id', range(1, len(fact_booking) + 1))

fact_booking.head()

/var/folders/xk/8b3j7bgd4r76vcr32987sg4r0000gn/T/ipykernel_3262/2925959835.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_booking['arrival_date'] = pd.to_datetime(
/var/folders/xk/8b3j7bgd4r76vcr32987sg4r0000gn/T/ipykernel_3262/2925959835.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_booking['date_id'] = fact_booking['arrival_date'].map(date_mapping)


,booking_id,date_id,reserved_room_type,assigned_room_type,is_canceled,lead_time,booking_changes,adr,stays_in_weekend_nights,stays_in_week_nights,reservation_status_id,guest_id
hotel,,,,,,,,,,,,
Resort Hotel,1,1,C,C,0,342,3,0.0,0,0,1,1
Resort Hotel,2,1,C,C,0,737,4,0.0,0,0,1,32970
Resort Hotel,3,1,A,C,0,7,0,75.0,0,1,1,35130
Resort Hotel,4,1,A,A,0,13,0,75.0,0,1,1,4
Resort Hotel,5,1,A,A,0,14,0,98.0,0,2,1,5


## 3.0 Load Dataframes

#### Define Connection Variables for MySQL Server and Database

In [23]:
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "WebPassa11!"
dst_dbname = "hotel_booking_dw"


#### Define Functions to Get Data and Set Data Into Databases (code taken from lab 4)

In [24]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(text(sql_query), connection);
    connection.close()
    
    return dframe
    
# makes connection and then determine whether we want to insert or update. 
def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")) # goes to col that you specify and turn it into a pkey

            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append') # append to existing df
    
    connection.close()



In [25]:
# create new data warehouse db
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(text(f"DROP DATABASE IF EXISTS `{dst_dbname}`;"))
connection.execute(text(f"CREATE DATABASE `{dst_dbname}`;"))
connection.execute(text(f"USE {dst_dbname};"))

# connection successful if no error messages
connection.close()

In [26]:
# load dataframes into data warehouse
db_operation = "insert"
# 4 sets; set contains what i need to call the set dataframe method below
tables = [('dim_guest', dim_guest, 'guest_id'),
          ('dim_date', dim_date, 'date_id'),
          ('dim_reservation_status', dim_reservation_status, 'reservation_status_id'),
          ('fact_booking', fact_booking, 'booking_id')]

In [27]:
# upload dataframes 
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)
    # goes to db, use all those dfs, and magically creates all the dimension tables

# 4.0 Validate Functionality 

In [28]:
# select data from dim_guest table
sql_guests= "SELECT * FROM hotel_booking_dw.dim_guest"
df_guests = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_guests)
df_guests.head()

,guest_id,name,email,phone-number,credit_card,customer_type,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled
0,1,Ernest Barnes,Ernest.Barnes31@outlook.com,669-792-1661,************4322,Transient,0,0,0
1,2,Andrea Baker,Andrea_Baker94@aol.com,858-637-6955,************9157,Transient,0,0,0
2,3,Rebecca Parker,Rebecca_Parker@comcast.net,652-885-2745,************3734,Transient,0,0,0
3,4,Laura Murray,Laura_M@gmail.com,364-656-8427,************5677,Transient,0,0,0
4,5,Linda Hines,LHines@verizon.com,713-226-5883,************5498,Transient,0,0,0


In [29]:
# select data from dim_date table 
sql_date = "SELECT * FROM hotel_booking_dw.dim_date"
df_date = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_date)
df_date.head()


,date_id,arrival_date_day_of_month,full_date,arrival_date_week_number,arrival_date_month,arrival_date_year
0,1,1,2015-07-01,27,July,2015
1,2,2,2015-07-02,27,July,2015
2,3,3,2015-07-03,27,July,2015
3,4,4,2015-07-04,27,July,2015
4,5,5,2015-07-05,28,July,2015


In [39]:
# select data from fact table; want to filter out observations w/ adr == 0 
# and want to look at full date info for when booking is on 2015-07-02	
sql_booking = """ 
    SELECT * FROM hotel_booking_dw.fact_booking fb 
    INNER JOIN hotel_booking_dw.dim_date dd
    ON fb.date_id = dd.date_id
    WHERE fb.date_id = 2 AND adr > 0;
"""
    
df_booking = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_booking)
df_booking.head()

,booking_id,date_id,reserved_room_type,assigned_room_type,is_canceled,lead_time,booking_changes,adr,stays_in_weekend_nights,stays_in_week_nights,reservation_status_id,guest_id,date_id,arrival_date_day_of_month,full_date,arrival_date_week_number,arrival_date_month,arrival_date_year
0,34,2,A,C,0,69,0,65.5,2,4,1,34,2,2,2015-07-02,27,July,2015
1,35,2,D,D,1,45,0,108.8,1,3,2,93472,2,2,2015-07-02,27,July,2015
2,36,2,D,D,1,40,0,108.8,1,3,2,36,2,2,2015-07-02,27,July,2015
3,37,2,A,C,0,15,0,98.0,1,3,1,114633,2,2,2015-07-02,27,July,2015
4,38,2,D,D,0,36,0,108.8,1,3,1,38,2,2,2015-07-02,27,July,2015


In [31]:
# perform aggregation -- use group by
# want to see what the average lead time based on arrival date is
# each date_id corresponds to a unique full-date of the guest's date of arrival
sql_aggregate = "SELECT date_id AS arrival_date, AVG(lead_time) AS average_lead_time FROM hotel_booking_dw.fact_booking GROUP BY date_id"
df_aggregate = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_aggregate)
df_aggregate.head()

,arrival_date,average_lead_time
0,1,186.7459
1,2,153.1290
2,3,72.8571
3,4,154.9659
4,5,83.6415


In [32]:
# group by assigned room type to get maximum adr based on assigned room type
sql_max_adr = "SELECT assigned_room_type, MAX(adr) AS max_average_daily_rate FROM hotel_booking_dw.fact_booking GROUP BY assigned_room_type"
df_max_adr = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_max_adr)
df_max_adr.head()

,assigned_room_type,max_average_daily_rate
0,C,508.0
1,A,5400.0
2,D,375.5
3,E,451.5
4,G,510.0
